In [6]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from openpyxl import Workbook
import requests
import time
import sys
import os
from selenium.webdriver.chrome.options import Options


print("웹드라이버 설정 시작")
path = ChromeDriverManager().install()
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)
print("웹드라이버 설정 완료")



search_name = input("1.공고명으로 검색할 키워드는 무엇입니까?: ")
date_start = input("2.조회 시작일자 입력(예:2019/01/01): ")
date_end = input("3.조회 종료일자 입력(예:2019/03/31): ")
folder_path = input("4.파일로 저장할 폴더 이름을 입력하세요(예:c\py_temp\): ")

save_txt = os.path.join(folder_path, "result.txt")
save_csv = os.path.join(folder_path, "result.csv")
save_xls = os.path.join(folder_path, "result.xlsx")


if not os.path.exists(folder_path):
    print(f"입력하신 경로에 {folder_path} 가 존재하지 않아 생성 후 크롤링 작업을 진행합니다.")
    os.makedirs(folder_path)
else:
    print(f"입력한 경로인 {folder_path} 가 존재하어 바로 크롤링 작업을 시작하겠습니다.")


url = "https://www.g2b.go.kr/index.jsp"
driver.get(url)
driver.implicitly_wait(5)


searchbar = driver.find_element(By.ID, "bidNm")
searchbar.clear()
searchbar.send_keys(search_name)

s_date = driver.find_element(By.ID, "fromBidDt")
s_date.send_keys(date_start)

e_date = driver.find_element(By.ID, "toBidDt")
e_date.send_keys(date_end)

driver.find_element(By.CLASS_NAME, "btn_dark").click()


driver.implicitly_wait(3)



driver.switch_to.default_content()  
main_frame = driver.find_element(By.ID, 'sub')
driver.switch_to.frame(main_frame)  
content_frame = driver.find_element(By.NAME, 'main')
driver.switch_to.frame(content_frame) 



full_html = driver.page_source
soup = bs(full_html, 'html.parser')
contents_list = soup.find('tbody')
rows = contents_list.find_all("tr")


orig_stdout = sys.stdout
f = open(save_txt, 'a', encoding='UTF-8')
sys.stdout = f


wb = Workbook()
ws = wb.active


rows = rows[:10]
i = 1


thead = soup.find('thead')
header_row = thead.find('tr')
header_cells = header_row.find_all('th')
header_values = [cell.get_text(strip=True) for cell in header_cells]

ws.cell(row=1, column=2, value=header_values[0])
ws.cell(row=1, column=3, value=header_values[1])
ws.cell(row=1, column=4, value=header_values[2])
ws.cell(row=1, column=5, value=header_values[3])
ws.cell(row=1, column=6, value="공고URL주소")
ws.cell(row=1, column=7, value=header_values[4])
ws.cell(row=1, column=8, value=header_values[5])
ws.cell(row=1, column=9, value=header_values[6])
ws.cell(row=1, column=10, value=header_values[7])
ws.cell(row=1, column=11, value=header_values[8])
ws.cell(row=1, column=12, value=header_values[9])


for row in rows:
    columns = row.find_all('td')

    first_column = columns[0].get_text(strip=True)
    second_column = columns[1].get_text(strip=True)
    third_column = columns[2].get_text(strip=True)
    fourth_column = columns[3].get_text(strip=True)
    fourth_column_href = columns[3]
    fifth_column = columns[4].get_text(strip=True)
    sixth_column = columns[5].get_text(strip=True)
    seventh_column = columns[6].get_text(strip=True)
    eighth_column = columns[7].get_text(strip=True)
    ninth_column = columns[8].get_text(strip=True)
    tenth_column = columns[9].get_text(strip=True)

    href = fourth_column_href.find('a')
    if href is not None:
        href_text = href.get_text(strip=True)
        href_url = href['href']

    print(f"{i}번째 공고내용을 출력합니다. ")
    print("업무:", first_column)
    print("공고번호-차수:", second_column)
    print("분류:", third_column)
    print("공고명: ", fourth_column)
    print("URL 주소: ", href_url)
    print("공고기관: ", fifth_column)
    print("수요기관: ", sixth_column)
    print("계약방법: ", seventh_column)
    print("입력일시(입찰마감일시): ", eighth_column)
    print("공동수급: ", ninth_column)
    print("투찰여부: ", tenth_column)
    print("\n")

    i += 1

    ws.cell(row=i, column=1, value=i)
    ws.cell(row=i, column=2, value=first_column)
    ws.cell(row=i, column=3, value=second_column)
    ws.cell(row=i, column=4, value=third_column)
    ws.cell(row=i, column=5, value=fourth_column)
    ws.cell(row=i, column=6, value=href_url)
    ws.cell(row=i, column=7, value=fifth_column)
    ws.cell(row=i, column=8, value=sixth_column)
    ws.cell(row=i, column=9, value=seventh_column)
    ws.cell(row=i, column=10, value=eighth_column)
    ws.cell(row=i, column=11, value=ninth_column)
    ws.cell(row=i, column=12, value=tenth_column)


sys.stdout = orig_stdout
f.close()


wb.save(save_csv)


wb.save(save_xls)

print("크롤링 작업을 완료하였습니다.")
driver.close()

웹드라이버 설정 시작


The version of chrome cannot be detected. Trying with latest driver version


웹드라이버 설정 완료
1.공고명으로 검색할 키워드는 무엇입니까?: 철강
2.조회 시작일자 입력(예:2019/01/01): 2019/01/01
3.조회 종료일자 입력(예:2019/03/31): 2023/06/06
4.파일로 저장할 폴더 이름을 입력하세요(예:c\py_temp\): D:\dava\


NameError: name 'file_path' is not defined